### Day 8

In [1]:
# Load input
with open("inputs/day8_input.dat","r") as f:
    data = f.read().splitlines()
signals = []
outputs = []
for line in data:
    ins,outs = line.split(' | ')
    signals.append(ins.split(' '))
    outputs.append(outs.split(' '))
    
# data = data[0].split(',')
# data = [int(x) for x in data]

In [2]:
print(signals[0:2])
print(outputs[0:2])

[['ec', 'cabfe', 'afebd', 'dbagef', 'afbcg', 'feabcd', 'cdef', 'eafdcbg', 'ecb', 'caegdb'], ['acfdg', 'ea', 'baec', 'dgbafec', 'cfebg', 'efa', 'cfebgd', 'dfbaeg', 'egcaf', 'abecfg']]
[['ceb', 'ecb', 'febac', 'ec'], ['bgcdfe', 'ebac', 'cadgf', 'dceagfb']]


### Part 1:
- Letters in each row point to a random part of a seven-segment-display (i.e. like a digital clock digit) that is lit up.
- The ordering of e.g. a=top, b=bottom is consistent across a line but different on each line.
- Need to count how many times a 1,4,7 or 8 appears in the puzzle input

Thoughts:
- Number of segments per number:
    - 0 = 6 segments on
    - 1 = 2 segments on # unique
    - 2 = 5 segments on
    - 3 = 5 segments on
    - 4 = 4 segments on
    - 5 = 5 segments on
    - 6 = 6 segments on
    - 7 = 3 segments on # unique
    - 8 = 7 segments on
    - 9 = 6 segments on
- If we have x number of segments, what number could it be?
    - 2 -> 1
    - 3 -> 7
    - 4 -> 4
    - 5 -> 2,3,5
    - 6 -> 0,6,9
    - 7 -> 8
- So 1,4,7,8 are unique, no need to actually solve them

In [3]:
#
count = 0
for row in outputs:
    for digit in row:
        if len(digit) in [2,3,4,7]:
            count += 1
print(count)

495


### Part 2:
- Solve all of them
- What is the sum of all of the output values?

Thoughts:
- Do it by hand once then program those exact steps?
- How many possibilities for each one?
    - 7 * 6 * 5 * 4 * 3 * 2 * 1 = 5040
    - With 200 inputs that's only 1M checks...
- Probably quicker to brute force

In [4]:
# Write functions to decode, convert to a number and test a decoding sequence
# Use a vector to represent each segment being on or off
# decode_map is also a dictionary taking a letter to a segment position in (0,6)
sequences = {
    (1,1,1,0,1,1,1):0,
    (0,0,1,0,0,1,0):1,
    (1,0,1,1,1,0,1):2,
    (1,0,1,1,0,1,1):3,
    (0,1,1,1,0,1,0):4,
    (1,1,0,1,0,1,1):5,
    (1,1,0,1,1,1,1):6,
    (1,0,1,0,0,1,0):7,
    (1,1,1,1,1,1,1):8,
    (1,1,1,1,0,1,1):9
}

def decoded_to_num(decoded, sequences):
    try:
        return sequences[tuple(decoded)]
    except:
        return -1

def decode(signal, decode_map):
    vector = [0,0,0,0,0,0,0]
    for letter in signal:
        vector[decode_map[letter]] = 1
    return vector
    
def test_decode_map(signals, decode_map, sequences):
    for s in signals:
        # Decode it
        decoded = decode(s, decode_map)
        num = decoded_to_num(decoded, sequences)
        if num == -1:
            return False
    return True

In [5]:
# 7 short for loops are easier to write and understand than a recursive function
all_possible_decode_maps = []
letters = list(range(7))
for l1 in letters:
    for l2 in letters:
        if l2 == l1:
            continue
        for l3 in letters:
            if l3 in [l1,l2]:
                continue
            for l4 in letters:
                if l4 in [l1,l2,l3]:
                    continue
                for l5 in letters:
                    if l5 in [l1,l2,l3,l4]:
                        continue
                    for l6 in letters:
                        if l6 in [l1,l2,l3,l4,l5]:
                            continue
                        for l7 in letters:
                            if l7 in [l1,l2,l3,l4,l5,l6]:
                                continue
                            all_possible_decode_maps.append({"a":l1,"b":l2,"c":l3,"d":l4,"e":l5,"f":l6,"g":l7})
print(len(all_possible_decode_maps))

5040


In [6]:
test_signals = ["acedgfb","cdfbe","gcdfa","fbcad","dab","cefabd","cdfgeb","eafb","cagedb","ab"]
test_output = ["cdfeb","fcadb","cdfeb","cdbaf"]
for decode_map in all_possible_decode_maps:
    works = test_decode_map(test_signals,decode_map,sequences)
    if works:
        print("It works!")
        result = []
        for o in test_output:
            decoded = decode(o, decode_map)
            num = decoded_to_num(decoded, sequences)
            result.append(num)
        print(result)

It works!
[5, 3, 5, 3]


In [7]:
# Try all sequences for all of the inputs and collect the answer
output_results = []
running_sum = 0
for ix,row in enumerate(signals):
    for decode_map in all_possible_decode_maps:
        works = test_decode_map(row,decode_map,sequences)
        if works:
            result = []
            for o in outputs[ix]:
                decoded = decode(o, decode_map)
                num = decoded_to_num(decoded, sequences)
                result.append(num)
            running_sum += result[0]*1000+result[1]*100+result[2]*10+result[3]
            output_results.append(result)
            continue

In [8]:
running_sum

1055164